In [ ]:
%load_ext sql
%sql sqlite:///pokemon.sqlite
from utils import *

``` python
from django.db import models


class PokemonKind(models.Model):
    name = models.TextField(blank=True, null=True)
    type1 = models.IntegerField(blank=True, null=True)
    type2 = models.IntegerField(blank=True, null=True)


class Trainer(models.Model):
    name = models.TextField(blank=True, null=True)
    pokedex = models.ManyToManyField(PokemonKind, through='Pokemon')

    favorite_pokemon = models.ForeignKey(
        PokemonKind, models.SET_NULL, null=True, related_name='+')


class Pokemon(models.Model):
    trainer = models.ForeignKey(
        Trainer, models.CASCADE, related_name='pokemons')
    pokemon_kind = models.ForeignKey(PokemonKind, models.CASCADE)
    pokelevel = models.IntegerField(blank=True, null=True)

```

In [ ]:
def view_trainers(limit=None, show=False):
    trainers = Trainer.objects.all()
    if limit:
        trainers = trainers[:limit]
        
    render('trainers.html', context={'trainers': trainers}, show=show)

In [ ]:
# Visualizando o que a view retorna

view_trainers(limit=3, show=True)

In [ ]:
Trainer.objects.count()

#### Chronometrando seu código

``` python
import timeit as pytimeit

def timeit(func):
    total = pytimeit.timeit(func, number=1)
    print('Ran in:', total)
```

In [ ]:
timeit(view_trainers)

# Investigando a quantidade de consultas ao Banco de Dados

In [ ]:
# Objeto "connection"

from django.db import reset_queries
from django.db import connection

reset_queries()

print("Queries:", connection.queries)

Trainer.objects.count()

print("Queries:", connection.queries)

#### Contando queries

``` python
class count_queries:
    def __enter__(self):
        self.count = len(connection.queries)
        return self

    def __exit__(self, *args, **kwargs):
        res = len(connection.queries) - self.count
        print('Number of queries:', res)
```

In [ ]:
with count_queries():
    view_trainers(limit=100)

In [ ]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    trainers = Trainer.objects.all()[:100]
    
    for t in trainers:
        t.favorite_pokemon.name
        
pretty(m.queries[:10])

In [ ]:
trainers_query = m.queries[0]
print(trainers_query)

In [ ]:
%sql $trainers_query

In [ ]:
pokemon_query = m.queries[1]
print(pokemon_query)

In [ ]:
%sql $pokemon_query

In [ ]:
query = (
    'SELECT *'
    'FROM "Trainer" LEFT OUTER JOIN "PokemonKind"'
    'ON ("Trainer"."favorite_pokemon_id" = "PokemonKind"."id")'
    'LIMIT 100'
)
%sql $query

In [ ]:
with count_queries() as m:
    trainers = Trainer.objects.select_related('favorite_pokemon').all()[:100]
    
    for t in trainers:
        t.favorite_pokemon.name
        
print(m.queries[0])

In [ ]:
def view_trainers(limit=None):
    trainers = Trainer.objects.select_related("favorite_pokemon").all()
    
    if limit:
        trainers = trainers[:limit]
        
    render('trainers.html', context={'trainers': trainers})
    

with count_queries():
    view_trainers(limit=100)

In [ ]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    trainers = Trainer.objects.all()[:100]
    
    for t in trainers:
        for p in t.pokemons.all():
            p.pokelevel

In [ ]:
Trainer.objects.select_related('pokemons').all()

In [ ]:
with count_queries() as m:
    trainers = Trainer.objects.prefetch_related('pokemons').all()[:100]
    
    for t in trainers:
        for p in t.pokemons.all():
            p.pokelevel

In [ ]:
query = m.queries[0]
print(query)

In [ ]:
query = m.queries[1]
print(query)

In [ ]:
def view_trainers(limit=None):
    trainers = (
        Trainer.objects
        .select_related("favorite_pokemon")
        .prefetch_related('pokemons')
        .all()
    )
    
    if limit:
        trainers = trainers[:limit]
        
    render('trainers.html', context={'trainers': trainers})
    

with count_queries():
    view_trainers(limit=100)

In [ ]:
display_template('trainers.html')

In [ ]:
with count_queries() as m:
    trainers = (
        Trainer.objects
        .prefetch_related('pokemons', 'pokemons__pokemon_kind')
        .all()[:100]
    )
    
    for t in trainers:
        for p in t.pokemons.all():
            p.pokemon_kind.name

# Testando as melhorias

In [ ]:
def view_trainers():
    trainers = (
        Trainer.objects
        .select_related('favorite_pokemon')
        .prefetch_related('pokemons', 'pokemons__pokemon_kind')
        .all()
    )
        
    render('trainers.html', context={'trainers': trainers})
    

with count_queries():
    timeit(view_trainers)

# Delegando o trabalho para o Banco de dados

In [ ]:
def calculate_averages():
    trainers = (
        Trainer.objects
        .prefetch_related('pokemons')
        .all()
    )

    for trainer in trainers:
        pokemons = trainer.pokemons.all()
        
        trainer.avg_level = (
            sum([p.pokelevel for p in pokemons]) / len(pokemons)
        ) if pokemons else 0
        
    return trainers


timeit(calculate_averages)

with count_queries() as m:
    trainers = calculate_averages()

print()
print(f'Treinador: {trainers[0].name}')
print(f'Média de level: {trainers[0].avg_level}')
print()
print(m.queries)

In [ ]:
from django.db.models import Avg

def calculate_averages():
    return list(
        Trainer.objects
        .annotate(avg_level=Avg('pokemons__pokelevel'))
        .order_by('id').all()
    )

timeit(calculate_averages)

with count_queries() as m:
    trainers = calculate_averages()


print()
print(f'Treinador: {trainers[0].name}')
print(f'Média de level: {trainers[0].avg_level}')
print()
print(m.queries)

# Escrevendo Testes

1. Instancie um objeto da forma mais completa possível (atenção para relacionamentos)
2. Teste o número de queries ao banco de dados.
3. Instancie um novo objeto.
4. Teste o número de queries ao banco de dados permanece o mesmo.

``` python
from django.test import TestCase

class TestView(TestCase):
    
    def test_number_of_queries(self):
        trainer1 = Trainer.objects.create(name="New")
        charizard = PokemonKind.objects.get(id=333)
        Pokemon.objects.create(
            trainer=trainer1, 
            pokemon_kind=charizard,
        )
        
        with self.assertNumQueries(2):
            calcular_medias()
            
        trainer2 = Trainer.objects.create(name="Other")
        bulbasaur = PokemonKind.objects.get(id=222)
        Pokemon.objects.create(
            trainer=trainer2, 
            pokemon_kind=bulbasaur,
        )        
            
        with self.assertNumQueries(2):
            calcular_medias()
```